In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

In [ ]:
ruta_capa_jurisdiccion = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\_6_1_Jurisdiccion_CAR"

df_jurisdiccion = pd.DataFrame.spatial.from_featureclass(ruta_capa_jurisdiccion)
# Se implementa .loc para evitar alertas en la modificación del dataframe
df_jurisdiccion_columnas_estandarizadas = df_jurisdiccion.loc[:,['CODDANE','Departamen']]
df_jurisdiccion_columnas_estandarizadas.rename({'CODDANE':'codigo_municipio','Departamen':'nombre_departamento'}, axis = 1, inplace = True)
# Transformación de registros de mayúsculas a letra capital, esto por medio de la función capitalize del método str.
df_jurisdiccion_columnas_estandarizadas['nombre_departamento'] = df_jurisdiccion_columnas_estandarizadas['nombre_departamento'].str.capitalize()

In [ ]:
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente
# La columna creada se le registran los datos haciendo un substring a la columna -codigo_municipio-  
df_jurisdiccion_columnas_estandarizadas.loc[:,'codigo_departamento'] = df_jurisdiccion_columnas_estandarizadas['codigo_municipio'].str.slice(0, 2)
df_jurisdiccion_columnas_estandarizadas = df_jurisdiccion_columnas_estandarizadas.loc[:,['codigo_departamento','nombre_departamento']]

In [ ]:
df_departamentos_unificados = df_jurisdiccion_columnas_estandarizadas.groupby(['codigo_departamento','nombre_departamento']).agg({'codigo_departamento':'count', 'nombre_departamento':'count'})

# Se cambia el nombre de las columnas asociadas a los conteos (funcion -rename-)
df_departamentos_unificados.rename({'codigo_departamento':'conteo_codigo_departamento', 'nombre_departamento':'conteo_nombre_departamento'}, axis = 1, inplace = True)

# Se resetan los índices
df_departamentos_unificados_completa = df_departamentos_unificados.reset_index()

# Se estandariza la tabla final
df_departamentos = df_departamentos_unificados_completa[['codigo_departamento','nombre_departamento']]

In [ ]:
ruta_bd = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"

# --- PREPARACIÓN DEPARTAMENTO ---
nombre_capa_departamento = 'tbl_departamento'
ruta_capa_departamento = os.path.join(ruta_bd, nombre_capa_departamento)
df_departamentos.spatial.to_table(location=ruta_capa_departamento)
print("------------------------------ Se crea la capa -tbl_departamento- ------------------------------")